# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
# api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)

file_path = os.path.join('/Users/kimhyeonjun/Desktop/KT Aivle/Lecture/7th_mini_project/project/open_api_key.txt')

# 파일 열기 및 내용 읽기
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        api_key = file.read()
except FileNotFoundError:
    print('파일을 찾을 수 없습니다.')

os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

: 

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
import time

# 웹드라이버 경로 설정
driver_path = '/Users/kimhyeonjun/Desktop/KT Aivle/Lecture/7th_mini_project/project/chromedriver-mac-arm64/chromedriver'

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # 브라우저 창을 띄우지 않고 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 웹드라이버 서비스 설정
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 에러 방지위해 먼저 방문
url = 'https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056'

try:
    # URL로 이동
    driver.get(url)
    
    # 잠시 대기 (페이지 로드 대기)
    time.sleep(5)
    
    # 페이지 소스 가져오기
    page_source = driver.page_source
    extracted_data = []
    
    for i in range(1, 8):
        url = f'https://aivle.kt.co.kr/home/brd/faq/listJson?ctgrCd=&pageIndex={i}'
        driver.get(url)
        
        time.sleep(2)
        page_source = driver.page_source

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(page_source, 'html.parser')
        # JSON 데이터가 텍스트로 포함된 부분을 찾기
        pre_tag = soup.find('pre')
        if pre_tag:
            json_text = pre_tag.text
            data = json.loads(json_text)  # JSON 문자열을 파싱하여 Python 객체로 변환
            
            # 데이터 추출
            for item in data['returnList']:
                extracted_item = {
                    '질문': BeautifulSoup(item['atclTitle'], 'html.parser').get_text(strip=True).replace('\n', ' ').replace('\r', ' '),
                    '답변': BeautifulSoup(item['atclCts'], 'html.parser').get_text(strip=True).replace('\n', ' ').replace('\r', ' '),
                    '분류': BeautifulSoup(item['ctgrNm'], 'html.parser').get_text(strip=True).replace('\n', ' ').replace('\r', ' ')
                }
                extracted_data.append(extracted_item)
        else:
            print("JSON 데이터를 포함하는 <pre> 태그를 찾을 수 없습니다.")
    
    # 추출된 데이터 프레임 변환
    df = pd.DataFrame(extracted_data)

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 웹드라이버 종료
    driver.quit()

/var/folders/v8/8fn2gpyn5g91v4190567rqf80000gn/T/ipykernel_64981/2540843758.py:55: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  '분류': BeautifulSoup(item['ctgrNm'], 'html.parser').get_text(strip=True).replace('\n', ' ').replace('\r', ' ')
/var/folders/v8/8fn2gpyn5g91v4190567rqf80000gn/T/ipykernel_64981/2540843758.py:54: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  '답변': BeautifulSoup(item['atclCts'], 'html.parser').get_text(strip=True).replace('\n', ' ').replace('\r', ' '),


In [4]:
df

,질문,답변,분류
0,1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요?,KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다...,모집/선발
1,2) 35세 이상은 지원할 수 없나요?,"본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도...",모집/선발
2,3) 미취업자의 기준이 뭔가요?,"미취업자의 기준은 아래와 같습니다.1) 기간의 정함이 있는 근로인 경우,2) 고용보...",모집/선발
3,4) 직장인도 지원할 수 있나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",모집/선발
4,5) 아르바이트를 하고 있는데 지원할 수 있나요?,고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주...,모집/선발
...,...,...,...
63,64) AICE (기존 AIFB) 자격증이 있어야 유리한가요?,네. AICE (기존 AIFB) Associate 자격 취득자는 선발 시 우대하고 ...,기타
64,65) AICE 시험관련 문의는 어디에 해야하나요?,help@aice.study 으로 문의 하시기 바랍니다.,기타
65,66) 국민취업지원제도를 신청하고 싶습니다.,"국민취업지원제도 신청과 관련 사항은 거주지 관할 고용센터, 업무 대행 위탁기관으로 ...",기타
66,67) 국민취업지원제도를 안해도 교육 참여되나요?,국민취업제도를 신청하지 않아도 KT 에이블스쿨 교육에 참여하실 수 있습니다. 국...,기타


In [5]:
df.to_csv('aivle_qna.csv', encoding='utf-8', index=False)

In [6]:
data = pd.read_csv('aivle_qna.csv', encoding='utf-8')
data.head()

,질문,답변,분류
0,1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요?,KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다...,모집/선발
1,2) 35세 이상은 지원할 수 없나요?,"본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도...",모집/선발
2,3) 미취업자의 기준이 뭔가요?,"미취업자의 기준은 아래와 같습니다.1) 기간의 정함이 있는 근로인 경우,2) 고용보...",모집/선발
3,4) 직장인도 지원할 수 있나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",모집/선발
4,5) 아르바이트를 하고 있는데 지원할 수 있나요?,고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주...,모집/선발


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [7]:
# OpenAI의 "text-embedding-ada-002" 모델을 사용하여 텍스트 임베딩 객체를 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Chroma 데이터베이스 인스턴스를 생성하고, 임베딩 함수를 설정합니다.
# persist_directory="./db"는 데이터베이스 파일을 현재 디렉토리의 db 폴더에 저장하도록 지정합니다.
database = Chroma(persist_directory="./QnA_DB", embedding_function=embeddings)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [8]:
# 데이터프레임의 '질문', '답변', '분류' 열에서 텍스트를 가져와 Document 객체의 리스트로 변환
documents = [Document(page_content=f"질문: {row['질문']} 답변: {row['답변']} 분류: {row['분류']}") for _, row in data.iterrows()]

# 생성한 Document 객체들을 Chroma 데이터베이스에 추가하는 코드 (database 객체가 정의되어 있어야 함)
database.add_documents(documents)

['e8508837-e9fc-4db6-a99c-6acdea8217ff',
 'b7c65002-216a-49dc-ac6a-335fcb03d744',
 '54a43bee-153b-42ed-939d-b7ace66ea90f',
 '5118ca6c-c35d-4f89-8eff-1bbbb9ee8cdd',
 'a284d564-5eaf-4e17-8398-a279847abe08',
 '79b9a7c3-eb96-43c0-9ed2-3a7f75663b4c',
 '43963ab9-23b8-498b-94b5-568bf519f960',
 '76524a58-7442-4728-ac09-7edc61ba229b',
 '7fa4d335-4d32-4b9d-8fd9-dbb1fbef3000',
 '2ed711cf-dbeb-4735-a5eb-c278ec034143',
 'c1ac1d34-6df0-42cc-8cbf-0755249b104d',
 '3c19c081-2418-4e0f-8f1c-17b7d38db681',
 '3d751aee-4026-458f-87ed-cc184a07ee8c',
 'e63c6a90-882c-40ee-8cc8-4ce678d2123d',
 '975ca0b0-98c5-4fb4-b990-036e594dd5bd',
 'f64f8a9a-68d4-41de-9f60-ae1545919c9c',
 '22665766-2d54-4c9e-9e7d-584641de9e5d',
 '5712411f-2f29-4d4d-b769-492fcc3efe42',
 '4885746b-17ad-490a-b0ce-0a474056b3ee',
 '31975cf9-3e15-4160-ba7b-239f1e4222a8',
 'baf5a5d3-1d0e-43ae-8310-6ed180245320',
 'f18b721d-a321-4742-bf9a-e2cdca8367cf',
 'b270c15e-d263-4be8-8b74-b16d740445d6',
 '542f5f04-b965-4bd3-a0c5-495f55c498b3',
 '7e098e39-6ecd-

* 입력된 데이터 조회

In [9]:
database.get()

{'ids': ['10819512-ea7b-4b70-bd63-8641498104fc',
  '1b0ff2f4-f468-4faa-b219-bd71189bf7ac',
  '1bdf1d0e-a2b1-44ef-ae3c-6d931a70139f',
  '1cec84fe-8faf-4296-908e-29335ba9cd2d',
  '22665766-2d54-4c9e-9e7d-584641de9e5d',
  '25de014d-7b4d-40f9-bd25-5e184418a726',
  '274807a7-0760-4384-8e1d-f9092a874cbf',
  '2eb3b7a1-6430-4dee-a5b0-e1951b180e3d',
  '2ed711cf-dbeb-4735-a5eb-c278ec034143',
  '31975cf9-3e15-4160-ba7b-239f1e4222a8',
  '31deff75-1764-462c-9718-dcae80d01aea',
  '3b00aed7-6c1c-408e-b154-1d6e7b5a05c9',
  '3c19c081-2418-4e0f-8f1c-17b7d38db681',
  '3d751aee-4026-458f-87ed-cc184a07ee8c',
  '3ed085fb-4a16-48cf-98fe-d063a7c4c21d',
  '3f716664-2fc8-4ede-a8d8-3ed0f04b3d1b',
  '43963ab9-23b8-498b-94b5-568bf519f960',
  '441370eb-1c22-4e95-b37f-fb25c25c8f80',
  '4506e199-83e4-4dfe-9c88-fb536cce41f5',
  '4885746b-17ad-490a-b0ce-0a474056b3ee',
  '4e7303aa-80cc-4515-bbde-33c5ff6b6790',
  '4fb39e11-272a-47ca-a836-ace8cda89852',
  '503d5fc9-cc89-4ce5-96d2-b61845949d60',
  '5118ca6c-c35d-4f89-8eff-

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [ ]:
# DB 생성
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

# 테이블 생성
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

conn.commit()
conn.close()

: 

* 모델 선언

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# ---------------------------------------------------------------
# 아래 코드가 다시 실행되어야 메모리가 초기화 됨

# 대화 메모리 초기화
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

: 

* 모델 사용 및 이력 확인

In [ ]:
# 질문
query = input("질문입력")  # 질문할 문장

# 유사도 점수
result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
sim1 = round(result[0][1], 5)
sim2 = round(result[1][1], 5)
sim3 = round(result[2][1], 5)

# 현재 시간
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')

# 답변
result = qa(query)
answer = result["answer"]

# # history에 insert
conn = sqlite3.connect(path)

# # ② 작업 : to_sql
data = pd.DataFrame({'datetime': [dt], 'query': [query], 'sim1':[sim1], 'sim2':[sim2], 'sim3':[sim3], 'answer':[answer]})
data.to_sql('history', conn, if_exists='append', index=False)

# # ③ 연결 종료
conn.close()

: 

In [ ]:
# DB 입력 확인
conn = sqlite3.connect(path)
df = pd.read_sql('SELECT * FROM history', conn)
display(df)
conn.close()

: 